In [2]:
############----------------smoking-------------------
import os
from shutil import copyfile
import json

data_root = '/root/jinyfeng/datas/xining-proj-datas/smoking-dataset'

label_savefolder = os.path.join(data_root, 'annotations')
if not os.path.exists(label_savefolder):
    os.makedirs(label_savefolder)
train_imgfolder = os.path.join(data_root, 'train')
val_imgfolder = os.path.join(data_root, 'val')
if not os.path.exists(train_imgfolder):
    os.makedirs(train_imgfolder)
if not os.path.exists(val_imgfolder):
    os.makedirs(val_imgfolder)
    
imagepath = os.path.join(data_root, 'images-2')
labelpath = os.path.join(data_root, 'ann_xml')

filenames = os.listdir(imagepath)
print(len(filenames))
imglist_all = []
for filename in filenames:
    if os.path.splitext(filename)[1] == '.jpg' or os.path.splitext(filename)[1] == '.JPG':
        imglist_all.append(filename)
# imglist_train = []
# imglist_val = []
train_num = len(imglist_all)*2//3
imglist_train = imglist_all[:train_num+1]
imglist_val = imglist_all[train_num+1:]

print(len(imglist_all), len(imglist_train), len(imglist_val))


trainfile_json= os.path.join(label_savefolder, 'train.json')
valfile_json= os.path.join(label_savefolder, 'val.json')
    
train_json = open(trainfile_json, 'w')
val_json = open(valfile_json, 'w')
from xml.dom.minidom import parse
import xml.dom.minidom
image_info_list_train=[]
image_annos_list_train=[]
image_id_list_train=[]
cate_list=[]
count_id = 10000
for imgfile in imglist_train:
#     print(imgfile)
    source = os.path.join(imagepath, imgfile)
    target = os.path.join(train_imgfolder, imgfile)
    copyfile(source, target)
    name = imgfile.split(".")[0]
#     print(name)
    label_file = os.path.join(labelpath, name+'.xml')
#     print(label_file)
    DOMTree = xml.dom.minidom.parse(label_file)
    collection = DOMTree.documentElement
    
#     anno_info = collection.getAttribute("annotation")
    filename = collection.getElementsByTagName("filename")
    filename_val = filename[0].childNodes[0].data
#     print(filename_val)
    if '_' in filename_val:
        image_id = filename_val.split('_')[2].split('.')[0]
    else:
        image_id = filename_val.split('.')[0][-3:]
#         print(image_id)
    image_id_list_train.append(image_id)
    
    width = collection.getElementsByTagName("width")
    width_val = width[0].childNodes[0].data
    height = collection.getElementsByTagName("height")
    height_val = height[0].childNodes[0].data
#     print(image_id, width_val, height_val)
    image_dict={
        'file_name': imgfile,
        'height': int(height_val),
        'width': int(width_val),
        "id": int(image_id)
    }
    image_info_list_train.append(image_dict)
    
    object_info = collection.getElementsByTagName("object")
#     print(len(object_info))
    for obj in object_info:
        name = obj.getElementsByTagName("name")
        name_str = name[0].childNodes[0].data
#         print(name_str)
#         if 'smoke' in name_str and len(name_str)>5:
#             print(imgfile, len(name_str), name_str)
        if name_str=='smoke':
            xmin=obj.getElementsByTagName("xmin")
            xmin_val = float(xmin[0].childNodes[0].data)
            ymin=obj.getElementsByTagName("ymin")
            ymin_val = float(ymin[0].childNodes[0].data)
            xmax=obj.getElementsByTagName("xmax")
            xmax_val = float(xmax[0].childNodes[0].data)
            ymax=obj.getElementsByTagName("ymax")
            ymax_val = float(ymax[0].childNodes[0].data)
#             print(xmin_val, ymin_val, xmax_val, ymax_val)
            category_id = 1
            count_id += 1
            width = xmax_val-xmin_val
            height = ymax_val-ymin_val
            anno_dict = {
                "iscrowd": 0,
                "area": height*width,
                "image_id": int(image_id),
                "bbox":[xmin_val, ymin_val, width, height],
                "category_id": 1,
                "id": count_id
            }
            image_annos_list_train.append(anno_dict)

category_dict={
    "supercategory": "smoke",
    "id": 1,
    "name": "smoking"
}
cate_list.append(category_dict)

train_label_json = json.dumps(dict(images=image_info_list_train,
                               annotations=image_annos_list_train,
                                  categories=cate_list))
train_json.writelines(train_label_json)
train_json.close()


image_info_list_val=[]
image_annos_list_val=[]
image_id_list_val=[]
for imgfile in imglist_val:
#     print(imgfile)
    source = os.path.join(imagepath, imgfile)
    target = os.path.join(val_imgfolder, imgfile)
    copyfile(source, target)
    name = imgfile.split(".")[0]
#     print(name)
    label_file = os.path.join(labelpath, name+'.xml')
#     print(label_file)
    DOMTree = xml.dom.minidom.parse(label_file)
    collection = DOMTree.documentElement
    
#     anno_info = collection.getAttribute("annotation")
    filename = collection.getElementsByTagName("filename")
    filename_val = filename[0].childNodes[0].data
    #     print(filename_val)
    if '_' in filename_val:
        image_id = filename_val.split('_')[2].split('.')[0]
    else:
        image_id = filename_val.split('.')[0][-3:]
#         print(image_id)
    image_id_list_val.append(image_id)
    
    width = collection.getElementsByTagName("width")
    width_val = width[0].childNodes[0].data
    height = collection.getElementsByTagName("height")
    height_val = height[0].childNodes[0].data
#     print(filename_val, image_id, width_val, height_val)
    image_dict={
        'file_name': imgfile,
        'height': int(height_val),
        'width': int(width_val),
        "id": int(image_id)
    }
    image_info_list_val.append(image_dict)
    
    object_info = collection.getElementsByTagName("object")
#     print(len(object_info))
    for obj in object_info:
        name = obj.getElementsByTagName("name")
        name_str = name[0].childNodes[0].data
#         print(name_str)
        if name_str=='smoke':
            xmin=obj.getElementsByTagName("xmin")
            xmin_val = float(xmin[0].childNodes[0].data)
            ymin=obj.getElementsByTagName("ymin")
            ymin_val = float(ymin[0].childNodes[0].data)
            xmax=obj.getElementsByTagName("xmax")
            xmax_val = float(xmax[0].childNodes[0].data)
            ymax=obj.getElementsByTagName("ymax")
            ymax_val = float(ymax[0].childNodes[0].data)
#             print(xmin_val, ymin_val, xmax_val, ymax_val)
            category_id = 1
            count_id += 1
            width = xmax_val-xmin_val
            height = ymax_val-ymin_val
            anno_dict = {
                "iscrowd": 0,
                "area": height*width,
                "image_id": int(image_id),
                "bbox":[xmin_val, ymin_val, width, height],
                "category_id": 1,
                "id": count_id
            }
            image_annos_list_val.append(anno_dict)
            
val_label_json = json.dumps(dict(images=image_info_list_val,
                               annotations=image_annos_list_val,
                                categories=cate_list))
val_json.writelines(val_label_json)
val_json.close()


print(len(image_id_list_train))
print(len(set(image_id_list_train)))
print(len(image_id_list_val))
print(len(set(image_id_list_val)))

4856
4856 3238 1618
3238
3238
1618
1618


In [7]:
import os
from shutil import copyfile

data_root = '/root/jinyfeng/datas/xining-proj-datas/smoking-dataset'
total_labelpath = os.path.join(data_root, 'ann_txt-2')
yolo_labelpath = os.path.join(data_root, 'labels')
if not os.path.exists(yolo_labelpath):
    os.makedirs(yolo_labelpath)
train_labelpath = os.path.join(yolo_labelpath, 'train')
val_labelpath = os.path.join(yolo_labelpath, 'val')

train_imgpath = os.path.join(data_root, 'train')
val_imgpath = os.path.join(data_root, 'val')
train_images = os.listdir(train_imgpath)
val_images = os.listdir(val_imgpath)
print('len(train_images)=======', len(train_images), 'len(val_images)=======', len(val_images))
if not os.path.exists(train_labelpath):
    os.makedirs(train_labelpath)
if not os.path.exists(val_labelpath):
    os.makedirs(val_labelpath)
    
txt_filenames = os.listdir(total_labelpath)
print('len(txt_filenames)======', len(txt_filenames))
for filename in txt_filenames:
    if os.path.splitext(filename)[1] != '.txt':
        print(filename)
        continue
#     print(filename)
    source = os.path.join(total_labelpath, filename)
    newfilename = filename.split('.')[0]+'.jpg'
    if newfilename in train_images:
#         print(newfilename, 'in train_images')
        target = os.path.join(train_labelpath, filename)
        copyfile(source, target)
    elif newfilename in val_images:
#         print(newfilename, 'in val_images')
        target = os.path.join(val_labelpath, filename)
        copyfile(source, target)
    else:
        print(newfilename, 'not in train_images and val_images')

len(train_images)======= 3238 len(val_images)======= 1618
len(txt_filenames)====== 4856


In [7]:
import os
import numpy as np

# filename_list = '/root/jinyfeng/projects/insightface/recognition/arcface_torch/sensoro_person_info_yolov7face_feat.log'
# savepath = '/root/jinyfeng/datas/sensoro/sensoro_person_info.txt'

filename_list = '/root/jinyfeng/projects/insightface/recognition/arcface_torch/sensoro_quality_face.log'
savepath = '/root/jinyfeng/datas/sensoro/sensoro_quality_face.txt'

filename_list_read = open(filename_list, 'r')
filename_lines = filename_list_read.readlines()

file_w = open(savepath, 'w')

line_num=0
for filename in filename_lines:
    filename = filename.strip('\n')
    sub_str = '.jpg'
    if '.jpg' not in filename:
        continue
    
    filename = filename.split(' ')[1]
    file_w.writelines(filename+'\n')
#     print('idx_str, filename========', idx_str, filename)
    
filename_list_read.close()
file_w.close()
